# Time Series Modelling : Forecasting Exercises

09 February 2023


Using saas.csv :

    
    Forecast, plot and evaluate using each at least 4 of the methods we discussed:
        Last Observed Value
        Simple Average
        Moving Average
        Holt's Linear Trend
        Holt's Seasonal Trend
        Based on previous year/month/etc., this is up to you.

Bonus:

    Using the store item demand data, create a forecast of sales_total and quantity for 2018 using the Previous Cycle approach. .
    Predict 2018 total monthly sales for a single store and/or item by creating a model using prophet.
    Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
    Plot the 2018 monthly sales predictions.



In [1]:
# for presentation purposes
import warnings
warnings.filterwarnings("ignore")

# wrangle
from env import username, password, host
import os

# transform
import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
import seaborn as sns


# working with dates
from datetime import datetime

# modeling
import statsmodels.api as sm
from statsmodels.tsa.api import Holt, ExponentialSmoothing
np.random.seed(0)

# evaluate
from sklearn.metrics import mean_squared_error
from math import sqrt 

In [2]:
# importing the csv

saas = pd.read_csv('saas.csv')

In [3]:
saas.head(3)

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
538471,2016-07-31,1021019,5538471,0.0,0.0
610468,2016-12-31,1014827,5610468,3.0,15.0
228612,2015-01-31,1001370,5228612,0.0,0.0


In [4]:
saas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Month_Invoiced     790458 non-null  object 
 1   Customer_Id        790458 non-null  int64  
 2   Invoice_Id         790458 non-null  int64  
 3   Subscription_Type  790458 non-null  float64
 4   Amount             790458 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 30.2+ MB


In [5]:
# renaming awkward cols

saas = saas.rename(columns = {'Month_Invoiced':'month_invoiced', 'Customer_Id':'customer_id',
                              'Amount': 'amount'})

In [7]:
# dropping unnecessary cols

saas = saas.drop(columns = {'Invoice_Id', 'Subscription_Type'})

In [8]:
# change 'sale_date' to datetime format
saas['month_invoiced'] = pd.to_datetime(saas['month_invoiced'])
    
# setting 'sale_date' as index and sorting it
saas = saas.set_index('month_invoiced').sort_index()

In [11]:
saas.head(10)

,customer_id,amount
month_invoiced,,
2014-01-31,1000000,0.0
2014-01-31,1000001,10.0
2014-01-31,1000002,0.0
2014-01-31,1000003,0.0
2014-01-31,1000004,5.0
2014-01-31,1000005,10.0
2014-01-31,1000006,0.0
2014-01-31,1000007,0.0
2014-01-31,1000008,0.0


### 1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum.

In [13]:
# set train size to be 50% of total 

train_size = int(round(saas.shape[0] * 0.5))

train_size

395229

In [14]:
# set validate size to be 30% of total 

val_size = int(round(saas.shape[0] * 0.3))

val_size

237137

In [15]:
# set test size

test_size = int(round(saas.shape[0] * 0.2))

In [23]:
# train : data to the end of 'train_size'

train = saas[:train_size]

train.shape

(395229, 2)

In [18]:
# validate to the end
val_end_index = train_size + val_size

val_end_index

632366

In [22]:
# setting val

val = saas[train_size:val_end_index]
val.shape

(237137, 2)

In [21]:
# setting test

test = saas[val_end_index:]

test.shape

(158092, 2)